# Manual Implementation of Problem 2 (Regression)

## Setting up the data

### Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Loading & Splitting the dataset

In [3]:
df = pd.read_csv("California_Houses.csv")

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

train_data_size = int(0.7 * len(df))
validation_data_size = int(0.15 * len(df))
test_data_size = len(df) - train_data_size - validation_data_size

train_data = df[:train_data_size]
validation_data = df[train_data_size:train_data_size + validation_data_size]
test_data = df[train_data_size + validation_data_size:]

### Extracting features matrix & Target Vector

In [4]:
X = train_data.iloc[:, 1:13]
T = train_data.iloc[:, 0]
X_vectors = {}
for col in X.columns:
    X_vectors[col] = X[col].values.reshape(-1, 1)

X_matrix = np.hstack(list(X_vectors.values()))

n_samples = X_matrix.shape[0]
X_matrix = np.hstack((np.ones((n_samples, 1)), X_matrix))

print(X_matrix[0:5 , :])
print(T.head())

[[ 1.00000000e+00  1.68120000e+00  2.50000000e+01  1.50500000e+03
   3.67000000e+02  1.39200000e+03  3.59000000e+02  3.60600000e+01
  -1.19010000e+02  1.81940136e+05  2.33893528e+05  4.08714567e+05
   2.93550331e+05]
 [ 1.00000000e+00  2.53130000e+00  3.00000000e+01  2.94300000e+03
   6.97000000e+02  1.56500000e+03  5.84000000e+02  3.51400000e+01
  -1.19460000e+02  8.13812430e+04  1.64389285e+05  3.42981620e+05
   3.27407281e+05]
 [ 1.00000000e+00  3.48010000e+00  5.20000000e+01  3.83000000e+03
   1.14200000e+03  1.31000000e+03  9.63000000e+02  3.78000000e+01
  -1.22440000e+02  5.40917690e+02  5.62389566e+05  7.40893792e+05
   7.06538013e+04]
 [ 1.00000000e+00  5.73760000e+00  1.70000000e+01  3.05100000e+03
   5.05000000e+02  1.70500000e+03  4.95000000e+02  3.42800000e+01
  -1.18720000e+02  2.94347219e+04  5.06150028e+04  2.26155681e+05
   4.44127558e+05]
 [ 1.00000000e+00  3.72500000e+00  3.40000000e+01  2.35100000e+03
   4.40000000e+02  1.06300000e+03  4.28000000e+02  3.66200000e+01


## Model Design and caclulating W*

$w^* = (X^{\top} X)^{-1} (X^{\top} T)$.

In [5]:
left_term = np.linalg.inv(X_matrix.T @ X_matrix)
right_term = X_matrix.T @ T.values
W_star = left_term @ right_term